# 1.16 de mandataris die een verhinderde mandataris vervangt heeft een startdatum gelijk aan de einddatum van de verhinderde mandataris

We load Javascript libraries to i) retrieve data (bindings) from a bestuurseenheid, ii) validate the document, and iii) display the JSON result.

In [38]:
import { getBestuurseenheidUrl } from "./utils.ts";
import {  getBindingsFromTurtleContent, validateDocument, fetchDocument } from 'npm:@lblod/lib-decision-validation';
import { display } from "https://deno.land/x/display/mod.ts";

const bestuurseenheidUuid = '2fd8218dd4a404931eeaef879e33506ae998e38d4ca745620f4f6044c230dff3';
const bestuurseenheidUrl = getBestuurseenheidUrl(bestuurseenheidUuid);
const bestuurseenheidBindings = await fetchDocument(bestuurseenheidUrl);

Below, we can find the SPARQL template query and SHACL shape for the use case.
The Turtle string is converted into bindings.

In [35]:
const mandataris_1_query: string = `
    PREFIX besluit: <http://data.vlaanderen.be/ns/besluit#>
    PREFIX mandaat: <http://data.vlaanderen.be/ns/mandaat#>
    PREFIX org: <http://www.w3.org/ns/org#>
    PREFIX lmb: <http://lblod.data.gift/vocabularies/lmb/>
    PREFIX generiek: <https://data.vlaanderen.be/ns/generiek#>

    select DISTINCT ($this as ?this) ?value
    where {
        $this a mandaat:Mandataris ;
            org:holds ?mandaat ;
            mandaat:status <http://data.vlaanderen.be/id/concept/MandatarisStatusCode/c301248f-0199-45ca-b3e5-4c596731d5fe> ;
            mandaat:einde ?eindeVerhinderdeMandataris .

        FILTER NOT EXISTS {
            ?vervangendeMandataris a mandaat:Mandataris ;
                org:holds ?mandaat ;
                mandaat:start ?eindeVerhinderdeMandataris .
        }
       
        BIND ("Deze mandataris is verhinderd, maar heeft geen vervangende mandataris met startdatum gelijk aan de einddatum van deze verhinderde mandataris." as ?value)
    }`;
    
const mandataris_1_shape: string = `
@prefix sh: <http://www.w3.org/ns/shacl#> .

<http://example.org/mandataris_1_blueprint>
  a sh:NodeShape ;
  sh:targetClass <http://data.vlaanderen.be/ns/mandaat#Mandataris> ;
  sh:sparql [
		sh:select """${mandataris_1_query}""" ;
        sh:message 'Deze mandataris is verhinderd, maar heeft geen vervangende mandataris met startdatum gelijk aan de einddatum van deze verhinderde mandataris.'
] .`;

const mandataris_1_bindings: Bindings[] = await getBindingsFromTurtleContent(mandataris_1_shape);

To test the shape, we retrieved a subject from the triple store that is applicable. In this case, we choose a mandataris URI.
Then, the template query from above is applied on this subject.
Next, a Comunica URL is generated to debug the results of this validation.

In [36]:
// Test subject
const subject = 'http://data.lblod.info/id/mandatarissen/610B8C31B7632D000800006C';
const mandataris_1_subject_query = mandataris_1_query.replaceAll('$this', `<${subject}>`);

const mandataris_1_comunica = `https://query.linkeddatafragments.org/#datasources=${encodeURIComponent(bestuurseenheidUrl)}&query=${encodeURIComponent(mandataris_1_subject_query)}`;
console.log(mandataris_1_comunica);

https://query.linkeddatafragments.org/#datasources=http%3A%2F%2Flocalhost%3A8890%2Fsparql%2F%3Fdefault-graph-uri%3D%26query%3D%250A%2520%2520%2520%2520construct%2520%257B%250A%2520%2520%2520%2520%2520%2520%2520%2520%253Fs%2520%253Fp%2520%253Fo%2520.%250A%2520%2520%2520%2520%257D%250A%2520%2520%2520%2520where%2520%257B%250A%2520%2520%2520%2520%2520%2520%2520%2520graph%2520%253Chttp%253A%252F%252Fmu.semte.ch%252Fgraphs%252Forganizations%252F32ff6774dcd0547c842f7b065f7b1f9441fba0ea2b2586700a0426795787b2f5%252FLoketLB-mandaatGebruiker%253E%2520%257B%250A%2520%2520%2520%2520%2520%2520%2520%2520%2520%2520%2520%2520%253Fs%2520%253Fp%2520%253Fo%2520.%250A%2520%2520%2520%2520%2520%2520%2520%2520%257D%250A%2520%2520%2520%2520%257D%250A%2520%2520%2520%2520&query=%0A%20%20%20%20PREFIX%20besluit%3A%20%3Chttp%3A%2F%2Fdata.vlaanderen.be%2Fns%2Fbesluit%23%3E%0A%20%20%20%20PREFIX%20mandaat%3A%20%3Chttp%3A%2F%2Fdata.vlaanderen.be%2Fns%2Fmandaat%23%3E%0A%20%20%20%20PREFIX%20org%3A%20%3Chttp%3A%2F%2Fwww.w

The full validation on all mandatarissen happens below:

In [39]:
const mandataris_1_validationReport = await validateDocument(bestuurseenheidBindings, mandataris_1_bindings);
await display({ "application/json": mandataris_1_validationReport},{ raw: true });